# Introduction

## Présentation

Redis est une base de données NoSQL qui se classe dans la catégorie "data structure server". C'est  un système permettant de traiter, de manière très performante, des données sous la forme clés/valeurs. Cette performance est obtenue car les données sont stockées directement sur la mémoire RAM. Cela permet d'accéder très facilement aux données 

La clé permet de récupérer et d'identifier de manière unique la donnée. Redis ne permet pas de stocker des modèles de données très complexes il faut donc ruser pour créer une structure de données complète et complexe.

## Les données 



Les valeurs peuvent appartenir aux types suivants:

- chaînes de caractères
- compteurs numériques (les données numériques seront stockées dans la base sous la forme de chaîne de caractères)
- listes 
- sets ou sets ordonnées
- dictionnaires

## Installation

Pour installer la librairie de Python permettant d'intéragir avec Redis

`pip install redis`

ou si on utilise pipenv comme dans notre cas 

`pipenv install redis`

# Let's play

In [5]:
import redis

LOCAL = False

In [6]:
redis_client = redis.Redis(host='localhost', port=6379, decode_responses=True)

Pour tester la connexion on peut pinguer la base de données :

In [7]:
redis_client.ping()

True

Vous pouvez regarder toutes les configurations utilisé par Redis, beaucoup ne sont pas utiles à notre niveau mais vous pouvez voir par exemple, `database`, `maxclients`, `port` , `timeout` qui peuvent être utile de modifier pour différentes applications

In [8]:
redis_client.config_get()

{'repl-diskless-load': 'disabled',
 'dynamic-hz': 'yes',
 'lazyfree-lazy-expire': 'no',
 'slowlog-log-slower-than': '10000',
 'auto-aof-rewrite-min-size': '67108864',
 'timeout': '0',
 'lfu-decay-time': '1',
 'active-defrag-threshold-lower': '10',
 'syslog-ident': 'redis',
 'active-defrag-threshold-upper': '100',
 'cluster-slave-no-failover': 'no',
 'replica-lazy-flush': 'no',
 'set-max-listpack-value': '64',
 'tls-key-file': '',
 'auto-aof-rewrite-percentage': '100',
 'acllog-max-len': '128',
 'notify-keyspace-events': '',
 'syslog-enabled': 'no',
 'daemonize': 'no',
 'zset-max-listpack-entries': '128',
 'zset-max-ziplist-value': '64',
 'replica-priority': '100',
 'rdbchecksum': 'yes',
 'slaveof': '',
 'replica-announced': 'yes',
 'repl-diskless-sync': 'yes',
 'repl-backlog-ttl': '3600',
 'set-max-intset-entries': '512',
 'aof-load-truncated': 'yes',
 'no-appendfsync-on-rewrite': 'no',
 'bio_cpulist': '',
 'jemalloc-bg-thread': 'yes',
 'logfile': '',
 'acl-pubsub-default': 'resetchann

## Stockage clé valeurs

Pour stocker une valeur correspondant à une clé 

In [9]:
KEY = "name"
VALUE = "yourname"
redis_client.set(KEY, VALUE)

True

True montre que le stockage s'est bien déroulé. 

Pour récupérer la valeur stocké il suffit d'appeler la méthode `GET` sur cette clé

In [10]:
redis_client.get(KEY)

'yourname'

Vous pouvez voir que à l'aide du `b` que la réponse n'est pas exactement ce que l'on a stocké mais une représentation en bytes de notre chaine de charactères. 

In [50]:
assert redis_client.get(KEY) == VALUE, "Les valeurs ne correspondent pas"

Pour cela, en python, il faut appeler la méthode `decode()` pour transformer des bytes en string 

In [55]:
value_from_redis = redis_client.get(KEY)

# Check if the value is bytes and decode if necessary
if isinstance(value_from_redis, bytes):
    value_from_redis = value_from_redis.decode()

# Now value_from_redis is guaranteed to be a string
assert value_from_redis == VALUE, "les valeurs ne correspondent pas"


On peut préciser au client de décoder les réponses et de les transformer directement en strings

In [56]:
redis_client = redis.StrictRedis(host='localhost' if LOCAL else 'redis', port=6379, charset='utf-8', decode_responses=True)

In [57]:
assert redis_client.get(KEY) == VALUE

ConnectionError: Error 11001 connecting to redis:6379. getaddrinfo failed.

Les valeurs peuvent aussi être des entiers

In [11]:
KEY = "age"
VALUE = 20

redis_client.set(KEY, VALUE)

True

In [12]:
redis_client.get(KEY)

'20'

Vous pouvez alors utiliser la méthode `incr` pour incrémenter cet entier. Il faut faire attention puisque redis renvoie toujours les entiers comme des chaines de charactères.

In [13]:
redis_client.incr(KEY)

21

## Exemple 

In [14]:
from ipywidgets import widgets, interact
from IPython.display import display

In [15]:
KEY = "click_number"
DEFAULT_VALUE = 0

redis_client.set(KEY, DEFAULT_VALUE)

button = widgets.Button(description="Click me!")
def on_button_clicked(button):
    redis_client.incr(KEY)
    print(f"Le bouton à été cliqué {redis_client.get(KEY)} fois")
    button.description = "Click me more !"
    
button.on_click(on_button_clicked)
display(button)

Button(description='Click me!', style=ButtonStyle())

## Les Listes

Redis permet aussi de stocker des valeurs de type `list`, pour cela nous avons à notre disposition un ensemble de méthodes :  

- La méthode `LPUSH`  permet d'ajouter un élément en tête de list la complexité est de O(1).  
- La méthode `RPUSH` permet d'ajouter un élément en queue de list la complexité est de O(1).  
- La méthode `LINSERT` permet d'ajouter un élément avant ou après la première occurence d'une certaine valeur, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `LSET` permet de modifier la valeur correspondant à l'index spécifié, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `RPOP` permet de supprimer un élément en queue de list la complexité est de O(1).
- La méthode `LPOP` permet de supprimer un élément en tête de list la complexité est de O(1).
- La méthode `LINDEX` permet d'afficher un élément en fonction de son index.
- La méthode `LREM` permet de supprimer un élément en fonction de son index.
- La méthode `LLEN` permet de récupérer la taille d'une liste.

Nous allons toutes les tester.

In [16]:
redis_client.lpush('language_list', "kotlin")
redis_client.lpush('language_list', "python")

2

On peut aussi ajouter plusieurs élément en même temps dans une liste

In [17]:
redis_client.lpush('language_list', "java", "c++")

4

In [18]:
f"La taille de la liste est {redis_client.llen('language_list')}"

'La taille de la liste est 4'

In [19]:
INDEX = 2
f"L'élément à l'index {INDEX} taille de la liste est '{redis_client.lindex('language_list', INDEX)}'"

"L'élément à l'index 2 taille de la liste est 'python'"

Le client renvoie la taille de la liste venant d'être mise à jour. On peut afficher tous les éléments de la liste en les retirant un par un.

In [20]:
while(redis_client.llen('language_list')!=0):

    print(redis_client.lpop('language_list'))

c++
java
python
kotlin


## Les dictionnaires 

Redis permet de stocker assez facilement des dictionnaires, on peut comme cela stocker des structures de données plus complexes. Pour cela nous avons plusieurs méthodes : 

- La méthode `HMSET` permet d'ajouter un dictionnaire à une clé
- La méthode `HSET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGETALL` permet de récupérer toutes les clés valeurs d'un dictionnaire.
- La méthode `HKEYS` permet de récupérer toutes les clés d'un dictionnaire
- La méthode `HVALS` permet de récupérer toutes les valeurs d'un dictionnaire

In [21]:
doc = {
    "name":"yourname",
    "age":20,
    "city":"paris"
}

In [22]:
redis_client.hmset("user1", doc)

True

In [23]:
redis_client.hgetall("user1")

{'name': 'yourname', 'age': '20', 'city': 'paris'}

On peut modifier uniquement une clé du dictionnaire

In [24]:
redis_client.hset("user1", "city", "lyon")

0

In [25]:
redis_client.hgetall("user1")

{'name': 'yourname', 'age': '20', 'city': 'lyon'}

## Les pipelines

In [26]:
import timeit

Lorsque l'on appelle une méthode depuis le client python, chaqu'une d'entres elles fait un appel à Redis, ce qui peut prendre du temps. Les pipelines permettent de réduire ce temps en stockant toutes les opérations et en les applicant une seul fois sur la base. Cela permet d'augmenter grandement les performances en réduisant les appels et le passage par le réseau. 

In [27]:
N = 10000

In [28]:
def store_key_values(N):
    for i in range(N):
        redis_client.set(f"key_{i}", f"value_{i}")

In [29]:
%%timeit
store_key_values(N=N)

1.93 s ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
def store_key_values_with_pipelines(N):
    p = redis_client.pipeline()
    for i in range(N):
        p.set(f"key_{i}", f"value_{i}")

In [31]:
%%timeit
store_key_values_with_pipelines(N=N)

14.5 ms ± 294 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
On voit alors que l'on peut diviser le temps par 100. Dans vos différents métiers les performances sont souvent très importan

# Exercice

Utiliser redis pour gérer les commandes d'un restaurant de burgers, pour simplifier les choses les clients pourront choisir leurs ingrédients dans 3 types de produits :

- Pain: blanc, complet, aux céréales
- Garniture : salade, tomates , oignons, concombres
- Protéines : poulet, boeuf, agneau, tofu

Chaque ingrédient équivaut à un prix et un poids, avec chaque commande vous devrez calculer le poids et le prix que les clients devront régler. 

1) Stocker les informations concernant les ingrédients dans redis   
2) Créer la méthode permettant de créer aléatoirement un burger  
3) Créer la méthode qui permet d'empiler dans redis les prochaines commandes (Réfléchir au type de données utilisé)  
4) Créer la méthode permettant de calculer le poids et le prix du burger  
5) Créer une boucle pour dépiler, réaliser et servir les burgers.  

Pour vous aider voici quelques données 

In [32]:
pains = {
    "blanc":{
        "poid":100,
        "prix": 2
    },
    "complet":{
        "poid":90,
        "prix": 3
    },
    "cereales":{
        "poid":120,
        "prix": 4
    }
}

In [33]:
garniture = {
    "salade":{
        "poid":50,
        "prix": 1
    }, "tomates":{
        "poid":60,
        "prix": 2
    } , "oignons":{
        "poid":10,
        "prix": 1.5
    }, "concombres":{
        "poid":30,
        "prix": 3
    }
}

In [34]:
proteine = {
    "poulet":{
        "poid":100,
        "prix": 6
    }, "boeuf":{
        "poid":150,
        "prix": 5
    } , "agneau":{
        "poid":120,
        "prix": 7
    }, "tofu":{
        "poid":80,
        "prix": 8
    }
}

In [35]:
def make_burger():
    pass

def make_order():
    pass

def get_next_order():
    pass

def process_price_weight():
    pass

def process_burger():
    return None, None, None

In [36]:
while True:
    burger, price, weight = process_burger()
    print(f'Le burger {burger} de {weight} gr à été préparé et servi il faut régler {price}€')
    break

Le burger None de None gr à été préparé et servi il faut régler None€
